# JSSP comparação entre euristicas Classicas: tratamento dos dados 

In [5]:
# %pip install pandas
# %pip install seaborn
%pip install -r ../requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.6/27.6 MB 11.6 MB/s eta 0:00:00 MB/s eta 0:00:01:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [ortools]━━━ 3/4 [ortools]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [4]:

csv_filename = "simulated_annealing_results.csv"
df = pd.read_csv(csv_filename)
print("\nPrimeiras 5 linhas do CSV:")
print(df.head())

print(f"\nEstatísticas dos tempos de execução:")
print(f"Tempo médio: {df['execution_time'].mean():.4f}s")
print(f"Tempo mínimo: {df['execution_time'].min():.4f}s")
print(f"Tempo máximo: {df['execution_time'].max():.4f}s")

print(f"\nEstatísticas do fitness:")
print(f"Fitness médio: {df['fitness'].mean():.4f}")
print(f"Melhor fitness: {df['fitness'].min():.4f}")
print(f"Pior fitness: {df['fitness'].max():.4f}")


Primeiras 5 linhas do CSV:
      id  execution_time  fitness  \
0  15780        0.023511     10.0   
1  16289        0.023689     11.0   
2  16798        0.017370     11.0   
3  17307        0.017879     10.0   
4  17816        0.017860     10.0   

                                     solution_vector  
0  [0.8694768123943948, 0.21399559397541457, 0.31...  
1  [0.002084693912435154, 0.6919373008888614, 0.5...  
2  [0.00233236802752379, 0.8216689597439779, 0.97...  
3  [-0.21863423712469304, 0.9084528719029524, 0.9...  
4  [0.7153694250480912, 2.7456131935229364, 0.696...  

Estatísticas dos tempos de execução:
Tempo médio: 0.0182s
Tempo mínimo: 0.0164s
Tempo máximo: 0.0237s

Estatísticas do fitness:
Fitness médio: 10.3333
Melhor fitness: 10.0000
Pior fitness: 12.0000
